# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
        file_path_list = glob.glob(os.path.join(root,'*'))


/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

156


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
session.execute("""CREATE KEYSPACE IF NOT EXISTS sparkifydb WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")

#### Set Keyspace

In [7]:
session.set_keyspace('sparkifydb')

## Queries

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
session.execute("""CREATE TABLE IF NOT EXISTS song_length (sessionId text, artist_name text, itemInSession text,\
song_title text, song_length text, PRIMARY KEY(sessionId, itemInSession))""")

                    

In [9]:
# Load data from CSV files into song_length table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_length (sessionId, artist_name, itemInSession, song_title, song_len) "
        query = query + "VALUES (%s, %s, %s, %s, %s) IF NOT EXISTS "
        session.execute(query, (int(line[8]), line[0], int(line[3]), line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## Execute query on song_length table
result = session.execute("""SELECT artist_name, song_title, song_len FROM song_length WHERE sessionId = 338 AND itemInSession = 4;""")
    
for row in result:
    print (row.artist_name, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.3073


In [11]:
## Create table for query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

session.execute("""CREATE TABLE IF NOT EXISTS session_table (userId text, artist_name text, song_name text, first_name text,
last_name text, itemInSession text, sessionId text, PRIMARY KEY(userId, sessionId, song_name, itemInSession))""")
                    

In [12]:
# Load data into session_table from CSV
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_table (userId, sessionId, song_name, itemInSession, artist_name, first_name, last_name)"  
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s) IF NOT EXISTS "`
        session.execute(query, (int(line[10]), int(line[8]), line[9], int(line[3]), line[0], line[1], line[4]))

In [13]:
## Run query 2
result_2 = session.execute("""SELECT artist_name, song_name, first_name, last_name FROM session_table WHERE userId = 10 AND sessionId = 182;""")
    
for row in result_2:
    print (row.artist_name, row.song_name, row.first_name, row.last_name)

Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Down To The Bone Keep On Keepin' On Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz


In [14]:
## Create table for query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
session.execute("""CREATE TABLE IF NOT EXISTS song_listeners (first_name text, last_name text, song text, PRIMARY KEY(song, first_name, last_name));""")

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_listeners (song, first_name, last_name, userId)"  
        query = query + "VALUES (%s, %s, %s) IF NOT EXISTS "
        session.execute(query, (line[9], line[1], line[4], int(line[10])))

In [16]:
result_3 = session.execute("""SELECT first_name, last_name FROM song_listeners WHERE song = 'All Hands Against His Own';""")
    
for row in result_3:
    print (row.first_name, row.last_name)

Jacqueline Lynch
Sara Johnson
Tegan Levine


### Drop the tables before closing out the sessions

In [17]:
table_list = ['song_length', 'session_table', 'song_listeners']

for table in table_list:
    session.execute("""DROP TABLE {0};""".format(table))


### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()